In [1]:
import pandas as pd
import numpy as np
import time, os
from scipy import stats

In [2]:
def processData(in_data):   
    data = in_data.copy()
    
    X = []
    Y = []
    
    points1 = data[-1][-4]
    points2 = data[-1][-3]
    
    winmult1 = points1 / points2
    winmult2 = points2 / points1    
    
    for i in range(1, len(data)):
        player = ((i+1)%2)+1
        otherplayer = (i%2)+1
        
        turn = data[i, 0]
        tracks = data[i, 1:-8]
        prevtracks = data[i-1, 1:-8]
        p1cards, p2cards = data[i,-8:-6], data[i,-6:-4]
        p1cardsp, p2cardsp = data[i-1,-8:-6], data[i-1,-6:-4]
        p1points, p2points = data[i,-4], data[i,-3]
        p1points8, p2points8 = data[min(i+8,len(data)-1),-4], data[min(i+8,len(data)-1),-3]
        p1action, p2action = data[i,-2], data[i,-1]
        
        targtrack = ((prevtracks[1::2] - tracks[1::2]) != 0).astype(int)
        if player == 1:
            targaction = p1action
            targcolor = ((p1cardsp - p1cards) != 0).astype(int)
            
            itracks = tracks.copy()
            
            pointadv = np.log1p((p1points8 - p1points) * winmult1)
            
            X.append(np.concatenate([x.ravel() for x in 
                                 [turn,itracks,p1cards,p2cards,p1points,p2points,pointadv]]))
        elif player == 2:
            targaction = p2action
            targcolor = ((p2cardsp - p2cards) != 0).astype(int)
            
            itracks = tracks.copy()
            itracks[1::2] = itracks[1::2] * -1
            
            pointadv = np.log1p((p2points8 - p2points) * winmult2)
            
            X.append(np.concatenate([x.ravel() for x in 
                                 [turn,itracks,p2cards,p1cards,p2points,p1points,pointadv]]))
        
        if targaction == 2:
            targcolor = np.zeros(2)
        
        otargaction = np.zeros(2)
        if targaction >= 1:
            otargaction[int(targaction)-1] = 1
        
        Y.append(np.concatenate([x.ravel() for x in [targtrack,targcolor,otargaction]]))
            
        # X is collecting and passing through data for each turn, noteable changes are setting it up so
        # the player specific data is always in the same spot when being trained on, and adding the other
        # players cards, as in play its not possible to know which cards the opponent has chosen.
        
        # for Y, the targtrack tracks which track if any was targeted by a claim, targcolor tracks which color
        # is grabed for cards, and targaction splits the action into a one-hot of the possible choices.
        # all choices that need to be made are split into 1s and 0s for what actions to choose, although 
        # for card color it could be done in one with -1, 1.
        
        # pointadv is *ideally* going to be passed through the NN and used for weighting the loss function,
        # using similarity to the targets as a base, and the pointadv as a multiplier to give the weight to
        # actions that give the most points over the next few turns as well as the whole game
        
        # considering the simplicity of the game I assume that only one turn of information is enough to 
        # conclude the best decision, without something like destination cards to increase complexity.
        
    return np.array(X),np.array(Y)

In [3]:
FX=[]; FY=[]

for i in range(10):
    data = np.load(os.getcwd()+'/pythonGame/saves/save'+str(i)+'.npy')
    X, Y = processData(data)
    FX.append(X)
    FY.append(Y)
    
FX = np.array(FX)
FY = np.array(FY)

In [4]:
np.save(os.getcwd()+'/pythonGame/saves/FX.npy', FX)
np.save(os.getcwd()+'/pythonGame/saves/FY.npy', FY)

In [5]:
FX,FY

(array([array([[ 0.        , -3.        ,  0.        ,  5.        ,  0.        ,
         -2.        ,  0.        ,  4.        ,  0.        , -6.        ,
          0.        , -4.        ,  0.        ,  3.        ,  0.        ,
          3.        ,  0.        ,  3.        ,  0.        , -2.        ,
          0.        ,  2.        ,  0.        ,  0.        ,  0.        ,
          0.        ,  0.        ,  2.81254077],
        [ 0.        , -3.        , -0.        ,  5.        , -0.        ,
         -2.        , -0.        ,  4.        , -0.        , -6.        ,
         -0.        , -4.        , -0.        ,  3.        , -0.        ,
          3.        , -0.        ,  3.        , -0.        , -2.        ,
         -0.        ,  0.        ,  2.        ,  2.        ,  0.        ,
          0.        ,  0.        ,  1.26851133],
        [ 1.        , -3.        ,  0.        ,  5.        ,  0.        ,
         -2.        ,  0.        ,  4.        ,  0.        , -6.        ,
       